<a href="https://colab.research.google.com/github/Nadian-Ali/Temporary_Vid_classification/blob/main/TransformerClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/Projects/Video_Rec/' 

/content/drive/My Drive/Projects/Video_Rec


<h1> Define deep model </h1>


In [4]:
!pip install -q git+https://github.com/tensorflow/docs

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow_docs.vis import embed
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [6]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [7]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [21]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 20



In [29]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    classes = 2

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.0), loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model


# def run_experiment():
#     filepath = "/tmp/video_classifier"
#     checkpoint = keras.callbacks.ModelCheckpoint(
#         filepath, save_weights_only=True, save_best_only=True, verbose=1
#     )

#     model = get_compiled_model()
#     history = model.fit(
#         train_data,
#         train_labels,
#         validation_split=0.15,
#         epochs=EPOCHS,
#         callbacks=[checkpoint],
#     )

#     model.load_weights(filepath)
#     _, accuracy = model.evaluate(test_data, test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     return model

In [30]:

def train_model():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        # validation_split=0.15,
        batch_size = 1,
        validation_data = (val_data,val_labels),
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    return model

<h1>Load data</h1>

In [31]:
train_data   = np.load('resampled_train_dataset.npy')
val_data     = np.load('resampled_val_dataset.npy')
train_labels = np.load('train_labels.npy')
val_labels   = np.load('val_labels.npy')


In [32]:
model =  train_model()


Epoch 1/20
13/13 [==============================] - ETA: 0s - loss: 1.9379 - accuracy: 0.3846
Epoch 1: val_loss improved from inf to 0.91757, saving model to /tmp/video_classifier
13/13 [==============================] - 2s 89ms/step - loss: 1.9379 - accuracy: 0.3846 - val_loss: 0.9176 - val_accuracy: 0.5000
Epoch 2/20
13/13 [==============================] - ETA: 0s - loss: 0.6545 - accuracy: 0.6923
Epoch 2: val_loss improved from 0.91757 to 0.80171, saving model to /tmp/video_classifier
13/13 [==============================] - 1s 67ms/step - loss: 0.6545 - accuracy: 0.6923 - val_loss: 0.8017 - val_accuracy: 0.3333
Epoch 3/20
13/13 [==============================] - ETA: 0s - loss: 2.1822 - accuracy: 0.5385
Epoch 3: val_loss did not improve from 0.80171
13/13 [==============================] - 1s 62ms/step - loss: 2.1822 - accuracy: 0.5385 - val_loss: 0.8708 - val_accuracy: 0.5000
Epoch 4/20
13/13 [==============================] - ETA: 0s - loss: 1.4020 - accuracy: 0.4615
Epoch 4: va